# YOLOV4 OBJECT DETECTION

### STEP 1: DARKNET CLONE AND INSTALLATION

In [ ]:
#!unzip -d /content/drive/MyDrive /content/drive/MyDrive/yolov4.zip

In [ ]:
#!zip -r /content/gdrive/MyDrive/yolov4.zip /content/gdrive/MyDrive/yolov4

In [1]:
# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15308, done.
remote: Total 15308 (delta 0), reused 0 (delta 0), pack-reused 15308
Receiving objects: 100% (15308/15308), 13.69 MiB | 17.31 MiB/s, done.
Resolving deltas: 100% (10404/10404), done.


In [2]:
!cp /content/drive/MyDrive/Studies/Intern/yolov4/image.c /content/darknet/src

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile  #I will use opencv library in darknet
!sed -i 's/GPU=0/GPU=1/' Makefile #I will use cpu library in darknet
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile #cudnn a necessary tool for cpu
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

/content/darknet


In [ ]:
# verify CUDA
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [4]:
# make darknet (builds darknet so that you can then use the darknet executable file to run or train object detectors)
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -DCUDNN_HALF -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -DCUDNN_HALF -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
  

### STEP 2: LOADING THE DATASET WE HAVE PREPARED

In [ ]:
# this is where my datasets are stored within my Google Drive (I created a yolov4 folder to store all important files for custom training) 
#%cd darknet/
#!ls /content/drive/MyDrive/yolov4

ls: cannot access '/content/drive/MyDrive/yolov4': No such file or directory


In [ ]:
# copy over both datasets into the root directory of the Colab VM (comment out test.zip if you are not using a validation dataset)
#!cp /content/drive/MyDrive/yolov4/obj.zip ../
#!cp /content/drive/MyDrive/yolov4/test.zip ../

In [ ]:
# unzip the datasets and their contents so that they are now in /darknet/data/ folder
#!unzip ../obj.zip -d data/
#!unzip ../test.zip -d data/

Archive:  ../obj.zip
checkdir:  cannot create extraction directory: data
           File exists
Archive:  ../test.zip
checkdir:  cannot create extraction directory: data
           File exists


### **STEP 3: LET'S PREPARE THE REQUIRED FILES FOR THE TRAINING**

In this step, we will create the .cfg file, obj.data, obj.names and train.txt files required for the training.

download cfg to google drive and change its name                         
```
from shutil import copy2                                    
copy2("/content/darknet/cfg/yolov4-custom.cfg","/content/drive/MyDrive/yolov4")    
```      
you can also copy like this


In [ ]:
# download cfg to google drive and change its name
#%cd darknet/
#!cp cfg/yolov4-custom.cfg /content/drive/MyDrive/yolov4/yolov4-obj.cfg

Changes we need to make in our config file:

(The values given here are the recommended values for these variables.)

1. The batch value in our config file is the number of images we will import into our convolutional neural network in each iteration. Subdivision determines how many subdivisions we will divide each batch into. We can set the subdivision value to 16. **batch = 64 and subdivision 16.**

2. Then we can change the image size that will enter the model from the width and height sections. The max_batches value determines how many iterations our model will take. We can set our class number to x2000. Since we will train a model consisting of one class, we set the Max_batches value to 6000. **We equalize max_batches(2000 x number of classes trained). But the minimum we can do is 6000, so if you have one, two and three classes, it should be 6000.**

3. Then we change the value of the step to 80% or 90% of our max_batches value. I set the value of the step to 4800 which is 80% of 6000. **We make the values of the steps (80% of max_batches), (90% of max_batches).**

4. We replace the classes values under the [yolo] heading with the number of classes we train.

5. The Steps parameter is the number of iterations that the learning rate will be reduced to fit our model well. Finally, in our config file, we set the value of the class parameters to 3, which is our class number. We change the Filters parameters to (class + 5)x3. **In our case, this value is 18. We also equalize the filter's variables (number of classes to train + 5 )x3.**

In [5]:
# upload the custom .cfg back to cloud VM from Google Drive

!cp /content/drive/MyDrive/Studies/Intern/yolov4/yolov4-obj.cfg ./cfg

**-obj.names and obj.data**

Let's create a file named obj.names in our folder named yolov3 and write the names of your objects that we will train the file with.

E.G:
```
traffic sign
traffic light
```

In the same folder, we create a file with the name obj.data and write the directory where we will save the number of objects we will train, the addresses of the files named train.txt, text.txt and obj.names that we will use while training, and the weights we find as a result of the training.

E.G:
```
classes = 1
train = data/train.txt
valid = data/test.txt
names = data/obj.names
backup = /mydrive/yolov4/backup
```


upload the obj.names and obj.data files to cloud VM from Google Drive

In [6]:
!cp /content/drive/MyDrive/Studies/Intern/yolov4/obj.names ./data
!cp /content/drive/MyDrive/Studies/Intern/yolov4/obj.data  ./data

#### **-Train and Test Files**




**generate_train.py**



```
  import os
  image_files = []
  os.chdir(os.path.join("data", "obj"))
  for filename in os.listdir(os.getcwd()):
      if filename.endswith(".jpg"):
          image_files.append("data/obj/" + filename)
  os.chdir("..")
  with open("train.txt", "w") as outfile:
      for image in image_files:
          outfile.write(image)
          outfile.write("\n")
      outfile.close()
  os.chdir("..")
```

**generate_test.py**



```
  import os

  image_files = []
  os.chdir(os.path.join("data", "test"))
  for filename in os.listdir(os.getcwd()):
      if filename.endswith(".jpg"):
          image_files.append("data/test/" + filename)
  os.chdir("..")
  with open("test.txt", "w") as outfile:
      for image in image_files:
          outfile.write(image)
          outfile.write("\n")
      outfile.close()
  os.chdir("..")
```


upload the generate_train.py and generate_test.py script to cloud VM from Google Drive


In [ ]:
#!cp /content/drive/MyDrive/Studies/Intern/yolov4/generate_train.py ./
#!cp /content/drive/MyDrive/Studies/Intern/yolov4/generate_test.py ./

In [ ]:
#!python generate_train.py
#!python generate_test.py

In [ ]:
# verify that the newly generated train.txt and test.txt can be seen in our darknet/data folder
!ls data/

9k.tree     eagle.jpg	 imagenet.labels.list	   obj.names	     voc.names
coco9k.map  giraffe.jpg  imagenet.shortnames.list  openimages.names
coco.names  goal.txt	 labels			   person.jpg
dog.jpg     horses.jpg	 obj.data		   scream.jpg


### STEP 4: REDUCE THE WEIGHTS OF PRE-TRAINED CONVOLUTIONAL LAYERS

In this step, we download the used deep learning layers weights for the pre-trained yolov3. We do not have to perform this step, but starting the training with these weights will help the model we train to work more accurately and shorten the training time.

In [ ]:
#!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

### STEP 5: TRAINING
All the necessary files are ready, we can start the training.

Training will begin with the next command.

The duration of our training may vary depending on factors such as the number of photos in your data set, the quality of the photos, and the number of objects you train. Our loss value is important for the accuracy of our model. The lower our Loss value, the more accurate our model will work. We can run our model until the loss value stops decreasing and train the most accurate model possible according to our data set.

In [ ]:
# train your custom detector! (uncomment %%capture below if you run into memory issues or your Colab is crashing)
# %%capture
#!./darknet detector train data/obj.data cfg/yolov4-obj.cfg yolov4.conv.137 -dont_show -map

In [ ]:
# the graph of our training.
#imShow('chart.png')

If we don't like the weights, we can continue the training from where we left off.

In [ ]:
#!./darknet detector train data/obj.data cfg/yolov4-obj.cfg /content/drive/MyDrive/Studies/Intern/yolov4/backup/yolov4-obj_last.weights -dont_show

### STEP 6: USE OUR TRAINED MODEL


Our training is complete, now we can make recognition on the photos we want.

Let's look at the average loss value and percent accuracy of our model

In [ ]:
#!./darknet detector map data/obj.data cfg/yolov4-obj.cfg /content/drive/MyDrive/Studies/Intern/yolov4/backup/yolov4-obj_last.weights

#### **Let's run our model**

Let's test our model for a single image

In [ ]:
# define helper functions
def imShow(path): 
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to upload files
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# use this to download a file  
def download(path):
  from google.colab import files
  files.download(path)

In [ ]:
cp /content/drive/MyDrive/yolov4/test_pred.zip ../

In [ ]:
%cd /content

In [ ]:
!unzip /content/test_pred.zip 

In [ ]:
%cd /content/darknet/

In [ ]:
# need to set our custom cfg to test mode 
%cd cfg
!sed -i 's/batch=64/batch=1/' yolov4-obj.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' yolov4-obj.cfg
%cd ..

To predict and see individual images

To predict and see individual images

In [ ]:
# run your custom detector with this command (upload an image to your google drive to test, thresh flag sets accuracy that detection must be in order to show it)
!./darknet detector test data/obj.data cfg/yolov4-obj.cfg /content/drive/MyDrive/yolov4/backup/yolov4-obj_last.weights  /content/drive/MyDrive/images/cfc_000311.jpg -ext_output -dont_show -out  result.json -thresh 0.3 
imShow('predictions.jpg')


In [ ]:
import json

json_file = open('/content/darknet/result.json', 'r')#file reading process
json_dict=json.load(json_file)#Contents of json file converted to dict data type

In [ ]:
json_dict[0]["objects"][0]["relative_coordinates"]['center_x']

In [ ]:
import cv2
img= cv2.imread('/content/drive/MyDrive/images/cfc_000311.jpg')

In [ ]:
center_x=img.shape[1]*json_dict[0]["objects"][0]["relative_coordinates"]['center_x']
center_y=img.shape[0]*json_dict[0]["objects"][0]["relative_coordinates"]['center_y']
width=img.shape[1]*json_dict[0]["objects"][0]["relative_coordinates"]['width']
height=img.shape[0]*json_dict[0]["objects"][0]["relative_coordinates"]['height']

x_center=int(xmin + width()/2)


y_center=int(ymin + height()/2)

In [ ]:
new_width=(img.shape[1]*json_dict[0]["objects"][0]["relative_coordinates"]['width'])/2
new_height=(img.shape[0]*json_dict[0]["objects"][0]["relative_coordinates"]['height'])/2

In [ ]:
x_min=center_x-new_width
y_min=center_y-new_height

width=int(xmax-xmin)




height=int(ymax-ymin)

In [ ]:
x_max=width+x_min
y_max=height+y_min

In [ ]:
w=int(width)
h=int(height)

In [ ]:
crop_img = img[int(y_min):int(y_min)+h,int( x_min):int(x_min)+w]


In [ ]:
cv2.imwrite('/content/deneme/deneme.jpg',crop_img)

In [ ]:
# run your custom detector with this command (upload an image to your google drive to test, thresh flag sets accuracy that detection must be in order to show it)
!./darknet detector test data/obj.data cfg/yolov4-obj.cfg /content/drive/MyDrive/yolov4/backup/yolov4-obj_last.weights /content/drive/MyDrive/images/cfc_000311.jpg -thresh 0.3
imShow('predictions.jpg')

### **STEP 7: PREDICTION WITH TEST DATASET**

**To predict and save multiple images**

In [ ]:
cp /content/drive/MyDrive/Studies/Intern/yolov4/test_pred.zip ../

In [ ]:
!unzip /content/test_pred.zip 

Archive:  /content/test_pred.zip
   creating: test_pred/
  inflating: test_pred/1041305_cfc_000140.jpg  
  inflating: test_pred/1041307_cfc_000142.jpg  
  inflating: test_pred/1041315_cfc_000150.jpg  
  inflating: test_pred/1041316_cfc_000151.jpg  
  inflating: test_pred/1041318_cfc_000153.jpg  
  inflating: test_pred/1041320_cfc_000155.jpg  
  inflating: test_pred/1041324_cfc_000159.jpg  
  inflating: test_pred/1041325_cfc_000160.jpg  
  inflating: test_pred/22991_cfc_002387.jpg  
  inflating: test_pred/22999_cfc_002395.jpg  
  inflating: test_pred/23015_cfc_002411.jpg  
  inflating: test_pred/23039_cfc_002435.jpg  
  inflating: test_pred/23055_cfc_002451.jpg  
  inflating: test_pred/449312_cfc_002573.jpg  
  inflating: test_pred/449314_cfc_002575.jpg  
  inflating: test_pred/449316_cfc_002577.jpg  
  inflating: test_pred/449321_cfc_002582.jpg  
  inflating: test_pred/449359_cfc_002620.jpg  
  inflating: test_pred/449369_cfc_002630.jpg  
  inflating: test_pred/449370_cfc_002631.jpg  


In [ ]:
%cd /content/darknet/

/content/darknet


In [ ]:
# need to set our custom cfg to test mode 
%cd cfg
!sed -i 's/batch=64/batch=1/' yolov4-obj.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' yolov4-obj.cfg
%cd ..

/content/darknet/cfg
/content/darknet


In [7]:
!cp /content/drive/MyDrive/Studies/Intern/yolov4/backup/yolov4-obj_last.weights ../

In [ ]:
import glob
import os
image_path="../test_pred"
image_path_list = glob.glob(os.path.join(image_path, '*'))
image_path_list.sort()

In [ ]:
def detectionPredict(imageDir):
    import glob
    import os
    import cv2
    os.system("./darknet detector test data/obj.data cfg/yolov4-obj.cfg ../yolov4-obj_last.weights {} -thresh 0.3".format(imageDir))

In [ ]:
from constant import *
import tqdm
import torch
from preprocessing import tensorize_image, tensorize_mask, image_mask_check
import cv2
from train import *
from PIL import Image

for i in tqdm.tqdm(range(len(image_path_list))):
    batch_test = image_path_list[i:i+1]
    detectionPredict(batch_test[0])
    img=cv2.imread("predictions.jpg")
    predict_name=batch_test[0]
    if not os.path.exists("../predict_sign"):
      os.mkdir("../predict_sign")
    predict_path=predict_name.replace('test_pred', 'predict_sign')
    cv2.imwrite(predict_path,img.astype(np.uint8))

0it [00:00, ?it/s]


### **STEP 8: Multi Class Prediction**

In [8]:
!cp -r /content/drive/MyDrive/Studies/Intern/For_Colab/* ../

In [9]:
!cp -r /content/drive/MyDrive/Studies/Intern/models ../

In [10]:
import os
if not os.path.exists("/content/data/"):
  os.mkdir("/content/data/")

In [ ]:
!cp -r /content/drive/MyDrive/Studies/Intern/yolov4/test_pred.zip ../
!unzip -d /content/data/ /content/test_pred.zip 

Archive:  /content/test_pred.zip
   creating: /content/data/test_pred/
  inflating: /content/data/test_pred/1041305_cfc_000140.jpg  
  inflating: /content/data/test_pred/1041307_cfc_000142.jpg  
  inflating: /content/data/test_pred/1041315_cfc_000150.jpg  
  inflating: /content/data/test_pred/1041316_cfc_000151.jpg  
  inflating: /content/data/test_pred/1041318_cfc_000153.jpg  
  inflating: /content/data/test_pred/1041320_cfc_000155.jpg  
  inflating: /content/data/test_pred/1041324_cfc_000159.jpg  
  inflating: /content/data/test_pred/1041325_cfc_000160.jpg  
  inflating: /content/data/test_pred/22991_cfc_002387.jpg  
  inflating: /content/data/test_pred/22999_cfc_002395.jpg  
  inflating: /content/data/test_pred/23015_cfc_002411.jpg  
  inflating: /content/data/test_pred/23039_cfc_002435.jpg  
  inflating: /content/data/test_pred/23055_cfc_002451.jpg  
  inflating: /content/data/test_pred/449312_cfc_002573.jpg  
  inflating: /content/data/test_pred/449314_cfc_002575.jpg  
  inflating

In [ ]:
%cd /content/darknet/

/content/darknet


In [ ]:
# need to set our custom cfg to test mode 
%cd cfg
!sed -i 's/batch=64/batch=1/' yolov4-obj.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' yolov4-obj.cfg
%cd ..

/content/darknet/cfg
/content/darknet


In [ ]:
!cp /content/drive/MyDrive/Studies/Intern/yolov4/backup/yolov4-obj_last.weights ../

In [ ]:
import glob
import os
image_path="../test_pred"
image_path_list = glob.glob(os.path.join(image_path, '*'))
image_path_list.sort()

In [11]:
def Detection(imageDir):
    os.system("./darknet detector test data/obj.data cfg/yolov4-obj.cfg ../yolov4-obj_last.weights {} -ext_output -dont_show -out result.json -thresh 0.5"
              .format(imageDir))

In [ ]:
%cd /content/

/content


#### Predict

In [ ]:
import os
import glob
import torch
import tqdm
import cv2
from tqdm import tqdm_notebook
from preprocess import tensorize_image
import numpy as np
from constant import *
from train import *
from natsort import natsorted

#### PARAMETERS #####
cuda = True
test = True
predict_name = "test_pred"
fs_model_name = "Unet_2.pt"
line_model_name = "SegNet.pt"
fs_model_path = os.path.join(MODELS_DIR, fs_model_name)
line_model_path = os.path.join(MODELS_DIR, line_model_name)
input_shape = input_shape
#####################

if test:
    if not os.path.exists(TEST_PREDICT_DIR): 
      os.mkdir(TEST_PREDICT_DIR)
    test_input_path_list = glob.glob(os.path.join(TEST_DIR, "*"))
    test_input_path_list = natsorted(test_input_path_list)
    predict_path = os.path.join(TEST_PREDICT_DIR, predict_name.split(".")[0])
else:
    if not os.path.exists(PREDICT_DIR): 
      os.mkdir(PREDICT_DIR)
    predict_path = os.path.join(PREDICT_DIR, predict_name.split(".")[0])

if not os.path.exists(predict_path): 
    os.mkdir(predict_path)

# LOAD MODEL
fs_model = torch.load(fs_model_path)
#Remember that you must call model.eval() to set dropout and batch normalization layers to evaluation mode before running inference. 
#Failing to do this will yield inconsistent inference results.
fs_model.eval()

line_model = torch.load(line_model_path)
line_model.eval()

if cuda:
    fs_model = fs_model.cuda()
    line_model = line_model.cuda()



# PREDICT
def predict(fs_model, line_model, images):


    for image in tqdm_notebook(images):
        img = cv2.imread(image)
        batch_test = tensorize_image([image], input_shape, cuda)

        Detection(image)
        img = cv2.imread("predictions.jpg")
        
        fs_output = fs_model(batch_test)
        line_output = line_model(batch_test)
        fs_out = torch.argmax(fs_output, axis=1)
        line_out = torch.argmax(line_output, axis=1)

        
        fs_out_cpu = fs_out.cpu()
        line_out_cpu = line_out.cpu()
        
        fs_outputs_list  = fs_out_cpu.detach().numpy()
        line_outputs_list  = line_out_cpu.detach().numpy()
        
        fs_mask = np.squeeze(fs_outputs_list, axis=0)
        line_mask = np.squeeze(line_outputs_list, axis=0)
       
        fs_mask_uint8 = fs_mask.astype('uint8')
        line_mask_uint8 = line_mask.astype('uint8')
        
        fs_mask_resize = cv2.resize(fs_mask_uint8, ((img.shape[1]), (img.shape[0])), interpolation = cv2.INTER_CUBIC)
        fs_line_resize = cv2.resize(line_mask_uint8, ((img.shape[1]), (img.shape[0])), interpolation = cv2.INTER_NEAREST)
        
        
        #img_resize = cv2.resize(img, input_shape)
        mask_ind = fs_mask_resize == 1
        mask_ind = fs_line_resize == 1
        #copy_img = img_resize.copy()
        copy_img = img.copy()
        
        img[fs_mask_resize==1, :] = (255, 0, 125)
        img[fs_line_resize==1, :] = (0, 0, 255)
        img[fs_line_resize==2, :] = (38, 255, 255)
        
        opac_image = (img/2 + copy_img/2).astype(np.uint8)
        cv2.imwrite(os.path.join(predict_path, image.split("/")[-1]), opac_image)
        #print("mask size from model: ", mask.shape),
        #print("resized mask size: ", mask_resize.shape)

if __name__ == "__main__":
    predict(fs_model, line_model, test_input_path_list)

## **STEP 9: MULTI CLASS PREDICTION IN VIDEO DATA**

In [ ]:
!./darknet detector demo data/obj.data cfg/yolov4-obj.cfg /content/yolov4-obj_last.weights -dont_show /content/drive/MyDrive/Studies/Intern/test/videos/btest1.mp4 -thresh 0.4 -i 0 -out_filename /content/btest1_result.mp4

### Video to Frame

In [12]:
!cp /content/drive/MyDrive/Studies/Intern/test/videos/test4.mp4 /content/test4.mp4

In [14]:
import cv2
import os
import glob
# Opens the Video file
if not os.path.exists("/content/data/"): 
  os.mkdir("/content/data/")
if not os.path.exists("/content/data/test"): 
  os.mkdir("/content/data/test")
for f in glob.glob("/content/data/test/*"):
  os.remove(f)

cap = cv2.VideoCapture('/content/test4.mp4')
i = 0
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == False:
        break
    cv2.imwrite('/content/data/test/'+str(i)+'.jpg',frame)
    i+=1
 
cap.release()
cv2.destroyAllWindows()

### Predict

In [ ]:
%cd /content/

/content


In [ ]:
import os
import glob
import torch
import tqdm
import cv2
from tqdm import tqdm_notebook
from preprocess import tensorize_image
import numpy as np
from constant import *
from train import *
from natsort import natsorted

#### PARAMETERS #####
cuda = True
test = True
predict_name = "test4_predicts"
fs_model_name = "Unet_2.pt"
line_model_name = "SegNet.pt"
fs_model_path = os.path.join(MODELS_DIR, fs_model_name)
line_model_path = os.path.join(MODELS_DIR, line_model_name)
input_shape = input_shape
#####################

if test:
    if not os.path.exists(TEST_PREDICT_DIR): 
      os.mkdir(TEST_PREDICT_DIR)
    test_input_path_list = glob.glob(os.path.join(TEST_DIR, "*"))
    test_input_path_list = natsorted(test_input_path_list)
    predict_path = os.path.join(TEST_PREDICT_DIR, predict_name.split(".")[0])
else:
    if not os.path.exists(PREDICT_DIR): 
      os.mkdir(PREDICT_DIR)
    predict_path = os.path.join(PREDICT_DIR, predict_name.split(".")[0])

if not os.path.exists(predict_path): 
    os.mkdir(predict_path)

# LOAD MODEL
fs_model = torch.load(fs_model_path)
#Remember that you must call model.eval() to set dropout and batch normalization layers to evaluation mode before running inference. 
#Failing to do this will yield inconsistent inference results.
fs_model.eval()

line_model = torch.load(line_model_path)
line_model.eval()

if cuda:
    fs_model = fs_model.cuda()
    line_model = line_model.cuda()



# PREDICT
def predict(fs_model, line_model, images):


    for image in tqdm_notebook(images):
        img = cv2.imread(image)
        batch_test = tensorize_image([image], input_shape, cuda)
        
        fs_output = fs_model(batch_test)
        line_output = line_model(batch_test)
        fs_out = torch.argmax(fs_output, axis=1)
        line_out = torch.argmax(line_output, axis=1)

        
        fs_out_cpu = fs_out.cpu()
        line_out_cpu = line_out.cpu()
        
        fs_outputs_list  = fs_out_cpu.detach().numpy()
        line_outputs_list  = line_out_cpu.detach().numpy()
        
        fs_mask = np.squeeze(fs_outputs_list, axis=0)
        line_mask = np.squeeze(line_outputs_list, axis=0)
       
        fs_mask_uint8 = fs_mask.astype('uint8')
        line_mask_uint8 = line_mask.astype('uint8')
        
        fs_mask_resize = cv2.resize(fs_mask_uint8, ((img.shape[1]), (img.shape[0])), interpolation = cv2.INTER_CUBIC)
        fs_line_resize = cv2.resize(line_mask_uint8, ((img.shape[1]), (img.shape[0])), interpolation = cv2.INTER_NEAREST)
        
        
        #img_resize = cv2.resize(img, input_shape)
        mask_ind = fs_mask_resize == 1
        mask_ind = fs_line_resize == 1
        #copy_img = img_resize.copy()
        copy_img = img.copy()
        
        img[fs_mask_resize==1, :] = (255, 0, 125)
        img[fs_line_resize==1, :] = (0, 0, 255)
        img[fs_line_resize==2, :] = (38, 255, 255)
        
        opac_image = (img/2 + copy_img/2).astype(np.uint8)
        cv2.imwrite(os.path.join(predict_path, image.split("/")[-1]), opac_image)
        #print("mask size from model: ", mask.shape),
        #print("resized mask size: ", mask_resize.shape)

if __name__ == "__main__":
    predict(fs_model, line_model, test_input_path_list)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:57: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/1936 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


### Frame to Video

In [ ]:
import cv2
import numpy as np
import os
from os.path import isfile, join
from natsort import natsorted
from tqdm import tqdm_notebook

pathIn= '/content/data/test_predicts/test4_predicts/'
pathOut = '/content/test4_predicts.mp4'
fps = 24
frame_array = []
files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
#for sorting the file names properly
files = natsorted(files)
for i in tqdm_notebook(range(len(files))):
    filename=pathIn + files[i]
    #reading each files
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    
    #inserting the frames into an image array
    frame_array.append(img)
out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
for i in tqdm_notebook(range(len(frame_array))):
    # writing to a image array
    out.write(frame_array[i])
out.release()

In [ ]:
!cp /content/test4_predicts.mp4 /content/drive/MyDrive/Studies/Intern/test/predicts

# **Classification**

## Step1: Crop Sign

In [15]:
import json
import cv2

def crop_sign():
  global img
  global positions
  if not os.path.exists("/content/data/crop_image"):
    os.mkdir("/content/data/crop_image")

  for f in glob.glob("/content/data/crop_image/*"):
    os.remove(f)

  positions=[]

  json_file = open('/content/darknet/result.json', 'r')#file reading process
  json_dict=json.load(json_file)#Contents of json file converted to dict data type
  
  img= cv2.imread(json_dict[0]["filename"])

  for i,obj in enumerate(json_dict[0]["objects"]):
    if obj['class_id'] ==0:

      center_x=img.shape[1]*obj["relative_coordinates"]['center_x']
      center_y=img.shape[0]*obj["relative_coordinates"]['center_y']
      width=img.shape[1]*obj["relative_coordinates"]['width']
      height=img.shape[0]*obj["relative_coordinates"]['height']

      #x_center=int(xmin + width()/2)
      #y_center=int(ymin + height()/2)

      new_width=(img.shape[1]*obj["relative_coordinates"]['width'])/2
      new_height=(img.shape[0]*obj["relative_coordinates"]['height'])/2

      x_min=abs(center_x-new_width)
      y_min=abs(center_y-new_height)

      #width=int(xmax-xmin)
      #height=int(ymax-ymin)

      x_max=width+x_min
      y_max=height+y_min

      w=int(width)
      h=int(height)

      positions.append([int(x_min), int(y_min), int(x_max),int(y_max)])
      
      crop_img = img[int(y_min):int(y_min)+h,int(x_min):int(x_min)+w]
      ci_path=json_dict[0]["filename"].replace('test','crop_image')
      cv2.imwrite(ci_path[:-4]+"-"+str(i)+".jpg",crop_img)
  return positions

In [ ]:
positions

[[-1, 352, 16, 398]]

In [ ]:
crop_sign()

[[1, 352, 18, 398]]

In [ ]:
import glob
import os
for f in glob.glob("/content/full_predict/*"):
  os.remove(f)

## Step2: Class Pred

In [16]:
import os
import cv2
from PIL import Image
import numpy as np
from keras import models
import glob

def class_pred():
  global pred
  global meta_image
  if not os.path.exists("/content/data/meta_image/"):
    os.mkdir("/content/data/meta_image/")

  for f in glob.glob("/content/data/meta_image/*"):
    os.remove(f)

  crop_img=os.listdir('/content/data/crop_image')
  
  for f in crop_img:
    if f.startswith("."):
      crop_img.remove(f)
  
  crop_img.sort()
  data=[]
  for i in crop_img:
    image=cv2.imread("/content/data/crop_image/"+i)
    imageRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_fromarray = Image.fromarray(imageRGB, 'RGB')
    resize_image = image_fromarray.resize((30, 30))
    data.append(np.array(resize_image))

  X_test = np.array(data)
  X_test = X_test/255
  model=models.load_model('/content/models/sign_ford_model.h5')
  y = model.predict(X_test)
  pred=np.argmax(y,axis=1)

  for n,pr in enumerate(pred):
    meta_path='/content/data/Meta/'
    meta_image_path=meta_path+str(pr)+'.png'
    meta_image=cv2.imread(meta_image_path, cv2.IMREAD_UNCHANGED)
    cv2.imwrite('/content/data/meta_image/'+str(n)+'.jpg',meta_image)

# **FINAL**

In [ ]:
%cd /content/darknet

/content/darknet


In [17]:
!cp -r /content/drive/MyDrive/Studies/Intern/Meta /content/data

In [ ]:
import os
import glob
import torch
import tqdm
import cv2
from tqdm import tqdm_notebook
from preprocess import tensorize_image
import numpy as np
from constant import *
from train import *
from natsort import natsorted
from PIL import Image

#### PARAMETERS #####
cuda = True
test = True
predict_name = "test4_predicts"
fs_model_name = "Unet_2.pt"
line_model_name = "SegNet.pt"
fs_model_path = os.path.join(MODELS_DIR, fs_model_name)
line_model_path = os.path.join(MODELS_DIR, line_model_name)
input_shape = input_shape
#####################

if test:
    if not os.path.exists(TEST_PREDICT_DIR): 
      os.mkdir(TEST_PREDICT_DIR)


    test_input_path_list = glob.glob(os.path.join(TEST_DIR, "*"))
    test_input_path_list = natsorted(test_input_path_list)
    predict_path = os.path.join(TEST_PREDICT_DIR, predict_name.split(".")[0])
    for f in glob.glob(predict_path+"/*"):
      os.remove(f)
else:
    if not os.path.exists(PREDICT_DIR): 
      os.mkdir(PREDICT_DIR)
    predict_path = os.path.join(PREDICT_DIR, predict_name.split(".")[0])

if not os.path.exists(predict_path): 
    os.mkdir(predict_path)

# LOAD MODEL
fs_model = torch.load(fs_model_path)
#Remember that you must call model.eval() to set dropout and batch normalization layers to evaluation mode before running inference. 
#Failing to do this will yield inconsistent inference results.
fs_model.eval()

line_model = torch.load(line_model_path)
line_model.eval()

if cuda:
    fs_model = fs_model.cuda()
    line_model = line_model.cuda()



# PREDICT
def predict(fs_model, line_model, images):

    global x_meta
    for image in tqdm_notebook(images):
        img = cv2.imread(image)
        batch_test = tensorize_image([image], input_shape, cuda)

        Detection(image)
        img = cv2.imread("predictions.jpg")

        positions = crop_sign()

        if not len(positions) == 0:
          class_pred()
        
        fs_output = fs_model(batch_test)
        line_output = line_model(batch_test)
        fs_out = torch.argmax(fs_output, axis=1)
        line_out = torch.argmax(line_output, axis=1)

        
        fs_out_cpu = fs_out.cpu()
        line_out_cpu = line_out.cpu()
        
        fs_outputs_list  = fs_out_cpu.detach().numpy()
        line_outputs_list  = line_out_cpu.detach().numpy()
        
        fs_mask = np.squeeze(fs_outputs_list, axis=0)
        line_mask = np.squeeze(line_outputs_list, axis=0)
       
        fs_mask_uint8 = fs_mask.astype('uint8')
        line_mask_uint8 = line_mask.astype('uint8')
        
        fs_mask_resize = cv2.resize(fs_mask_uint8, ((img.shape[1]), (img.shape[0])), interpolation = cv2.INTER_CUBIC)
        fs_line_resize = cv2.resize(line_mask_uint8, ((img.shape[1]), (img.shape[0])), interpolation = cv2.INTER_NEAREST)
        
        
        #img_resize = cv2.resize(img, input_shape)
        mask_ind = fs_mask_resize == 1
        mask_ind = fs_line_resize == 1
        #copy_img = img_resize.copy()
        copy_img = img.copy()
        
        img[fs_mask_resize==1, :] = (255, 0, 125)
        img[fs_line_resize==1, :] = (0, 0, 255)
        img[fs_line_resize==2, :] = (38, 255, 255)
        
        opac_image = (img/2 + copy_img/2).astype(np.uint8)

        if not len(positions) == 0:
          for i, pos in enumerate(positions):
            x_meta=cv2.imread('/content/data/meta_image/'+str(i)+'.jpg', cv2.IMREAD_UNCHANGED)
            
            x_len = int(pos[2]-pos[0])
            y_len = int(pos[3]-pos[1])

            
            res_perc = int(min(x_len, y_len)*70/100)

            if res_perc >= 40:
              res_perc = 40

            x_meta = cv2.resize(x_meta, (res_perc, res_perc))
            
            
            #x = pos[2] - res_perc
            #y = pos[3] - res_perc
            x = pos[0]+2
            y = pos[1]+2

            '''
            y1, y2 = y, y + x_meta.shape[0]
            x1, x2 = x, x + x_meta.shape[1]

            alpha_s = x_meta[:, :, 3] / 255.0
            alpha_l = 1.0 - alpha_s

            for c in range(0, 3):
                opac_image[y1:y2, x1:x2, c] = (alpha_s * x_meta[:, :, c] +
                                          alpha_l * opac_image[y1:y2, x1:x2, c]) 
            '''

            opac_image[y:y+x_meta.shape[0],x:x+x_meta.shape[1]] = x_meta

        cv2.imwrite(os.path.join(predict_path, image.split("/")[-1]), opac_image)
        #print("mask size from model: ", mask.shape),
        #print("resized mask size: ", mask_resize.shape)

if __name__ == "__main__":
    predict(fs_model, line_model, test_input_path_list)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:62: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/2210 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [ ]:
x_meta.shape

(15, 15, 3)

In [ ]:
            x_len = int(pos[2]-pos[0])
            y_len = int(pos[3]-pos[1])

            if x_len<31 or y_len<31:
              res = min(x_len, y_len)
              x_meta = cv2.resize(x_meta, (res-5, res-5))
            else:
              x_meta = cv2.resize(x_meta, (30, 30))
              

### File Operations

In [ ]:
!cp /content/data/testx/569234_cfc_005061.jpg /content/data/test/
#!cp /content/data/test_pred/4060_cfc_004060.jpg /content/data/test/

In [ ]:
!cp -r /content/drive/MyDrive/Studies/Intern/For_Colab /content/Intern
!cp -r /content/drive/MyDrive/Studies/Intern/Meta /content/Intern
!cp -r /content/drive/MyDrive/Studies/Intern/models /content/Intern
!cp -r /content/drive/MyDrive/Studies/Intern/yolov4 /content/Intern

In [ ]:
!zip -r /content/test4_predicts.zip /content/data/test_predicts/test4_predicts/

  adding: content/data/test_predicts/test_final/ (stored 0%)
  adding: content/data/test_predicts/test_final/666488_cfc_000704.jpg (deflated 3%)
  adding: content/data/test_predicts/test_final/638936_cfc_001785.jpg (deflated 3%)
  adding: content/data/test_predicts/test_final/569162_cfc_004989.jpg (deflated 3%)
  adding: content/data/test_predicts/test_final/1041324_cfc_000159.jpg (deflated 2%)
  adding: content/data/test_predicts/test_final/449397_cfc_002658.jpg (deflated 3%)
  adding: content/data/test_predicts/test_final/666482_cfc_000698.jpg (deflated 3%)
  adding: content/data/test_predicts/test_final/488193_cfc_001542.jpg (deflated 3%)
  adding: content/data/test_predicts/test_final/449259_cfc_002520.jpg (deflated 3%)
  adding: content/data/test_predicts/test_final/567200_cfc_003027.jpg (deflated 3%)
  adding: content/data/test_predicts/test_final/510392_cfc_002355.jpg (deflated 2%)
  adding: content/data/test_predicts/test_final/569163_cfc_004990.jpg (deflated 2%)
  adding: cont

In [ ]:
!cp /content/test4_predicts.zip /content/drive/MyDrive/Studies/Intern/test/